# Cyfrowe przetwarzanie obrazów i sygnałów
## Ćwiczenie 4 
Celem ćwiczenia jest praktyczne wypróbowanie działania filtrów w celu wyeliminowania niepożądanych zakłóceń z sygnału EKG. Proszę wybrać rodzaj filtra do eksperymentowania, np. Butterwortha lub Czebyszewa. Do filtracji wykorzystać gotowe funkcje z biblioteki scipy.signal. Biblioteka posiada również funkcje wspomagające projektowanie filtrów, które można zastosować.
### Autorzy: 
 Kinga Marek 235 280,  
 Mariusz Wiśniewski 241 393

### Zadanie 1  

Wczytaj sygnał ekgnoise.txt i zauważ zakłócenia nałożone na sygnał. Wykreślić częstotliwościową charakterystykę amplitudową sygnału.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import mpld3
mpld3.enable_notebook()

fs = 360 
data = np.loadtxt("Data/ekg_noise.txt")
data = np.hsplit(data, 2)

plt.rcParams['figure.figsize']=[12,8]
plt.plot(data[0], data[1])
plt.xlabel("time [s]")
plt.ylabel("ECG [mV]")
plt.title("Wczytany sygnał")
plt.show()

fft_amplitude = np.abs(np.fft.fft(data[1]))
fft_freq = np.abs(np.fft.fftfreq(len(data[1]), 1/fs))

plt.rcParams['figure.figsize']=[10,8]
plt.plot(fft_freq, fft_amplitude)
plt.title('Signal frequency spectrum')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Amplitude')
plt.show()

### Zadanie 3

Zbadaj filtr dolnoprzepustowy o częstotliwości granicznej 60 Hz w celu redukcji zakłóceń pochodzących z sieci zasilającej.  Wyznacz parametry filtra, wykreśl jego charakterystykę (zależność tłumienia od częstotliwości), przebieg sygnału po filtracji oraz jego widmo.  Można też wyznaczyć różnicę między sygnałem przed i po filtracji i widmo tej różnicy.

In [ ]:
from scipy import signal

# wp and ws are normalized using Nyquist frequency (fs/2)
wp = 1/(fs/2) # passband frequency cutoff
ws = 60/(fs/2) # stopband frequency cutoff

# in decibels, loss and attenatuation in passband/stopband
gpass = 3
gstop = 15

# fit order and critical frequencies to the requirements
N, Wn = signal.cheb1ord(wp, ws, gpass, gstop, analog=False, fs=fs) 
# get the filter coefficients so we can check its frequency response.
b, a = signal.cheby1(N, 1, Wn, btype='lowpass')
# get the filter to filter the signal later
fil = signal.cheby1(N, 1, Wn, btype='lowpass', output = 'sos', fs=fs) 
# compute the frequency response of a digital filter
w, h = signal.freqz(b, a)

plt.plot(0.5*fs*w/np.pi, np.abs(h))
plt.plot(60, 0.5*np.sqrt(2), 'ko')
plt.title('Chebyshev I lowpass filter fit to constraints')
plt.xlabel('Normalized frequency')
plt.ylabel('Amplitude [dB]')
plt.grid(which='both', axis='both')
plt.show()

#plt.plot(w *(max(w)), abs(h))
#plt.title('Chebyshev I lowpass filter fit to constraints')
#plt.xlabel('Normalized frequency')
#plt.ylabel('Amplitude [dB]')
#plt.grid(which='both', axis='both')
#plt.show()

filtered = signal.sosfilt(fil, data[1])
plt.rcParams['figure.figsize']=[12,8]
plt.plot(data[0], filtered)
plt.xlabel("time [s]")
plt.ylabel("ECG [mV]")
plt.title("Wczytany sygnał")
plt.show()
plt.rcParams['figure.figsize']=[12,8]
plt.plot(data[0], data[1]-filtered)
plt.xlabel("time [s]")
plt.ylabel("ECG [mV]")
plt.title("Wczytany sygnał")
plt.show()

### Zadanie 3

Zastosuj następnie, do sygnału otrzymanego w punkcie 2, filtr górnoprzepustowyo częstotliwości granicznej 5 Hz w celu eliminacji pływania linii izoelektrycznej.Sporządź wykresy sygnałów jak w punkcie 2.


In [ ]:
# wp and ws are normalized using Nyquist frequency (fs/2)
wp = 5/(fs/2) # passband frequency cutoff
ws = 1/(fs/2) # stopband frequency cutoff

# in decibels, loss and attenatuation in passband/stopband
gpass = 3
gstop = 10

# fit order and critical frequencies to the requirements
N, Wn = signal.cheb1ord(wp, ws, gpass, gstop, analog=False, fs=fs) 
# design Chebyshev type 1 filter
b, a = signal.cheby1(N, 1, Wn, btype='highpass')
fil = signal.cheby1(N, 1, Wn, btype='highpass', output = 'sos', fs=fs) 
# compute the frequency response of a digital filter
w, h = signal.freqz(b, a)

plt.plot(w, 20*np.log10(abs(h)))
plt.title('Chebyshev I lowpass filter fit to constraints')
plt.xlabel('Normalized frequency')
plt.ylabel('Amplitude [dB]')
plt.grid(which='both', axis='both')
plt.show()

#plt.plot(w *(max(w)), abs(h))
#plt.title('Chebyshev I lowpass filter fit to constraints')
#plt.xlabel('Normalized frequency')
#plt.ylabel('Amplitude [dB]')
#plt.grid(which='both', axis='both')
#plt.show()

filtered2 = signal.sosfilt(fil, filtered)

fft_amplitude = np.abs(np.fft.fft(filtered2))
fft_freq = np.abs(np.fft.fftfreq(len(data[1]), 1/fs))

plt.rcParams['figure.figsize']=[10,8]
plt.plot(fft_freq, fft_amplitude)
plt.title('Signal frequency spectrum')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Amplitude')
plt.show()